In [52]:
from langchain.prompts import PromptTemplate

In [53]:
from langchain.pydantic_v1 import BaseModel
class ParameterList(BaseModel):
    parameters: list[str]

# Get parameters and values

In [54]:
from langchain.output_parsers.pydantic import PydanticOutputParser
parameter_output_parser = PydanticOutputParser(pydantic_object=ParameterList)

In [74]:
parameter_pre_prompt = PromptTemplate.from_template(
    """I am an architect and want to describe building products in detail. Please generate a list of 20 accurate, unique, and important Parameters for an omniclass value. The omniclass we will be creating data for is {product}.""",
)

parameter_format_prompt = PromptTemplate.from_template(
    """I am an architect and want to describe building products in detail.
    
    {parameters}
    
    {format_instructions}""",
    partial_variables={'format_instructions': parameter_output_parser.get_format_instructions()}
)

In [75]:
from dotenv import load_dotenv
load_dotenv()

True

In [57]:
from langchain.chat_models import ChatOpenAI

chat = ChatOpenAI(model_name='gpt-3.5-turbo')

In [76]:
from langchain.schema.output_parser import StrOutputParser
parameter_pre_chain = parameter_pre_prompt | chat | StrOutputParser()
parameter_chain = {'parameters': parameter_pre_chain} | parameter_format_prompt | chat | parameter_output_parser

In [77]:
product = '22-03 11 16 Architectural Cast-in Place Concrete Forming'
parameters = parameter_chain.invoke({"product": product})

In [78]:
parameters

ParameterList(parameters=['Formwork type', 'Formwork system', 'Formwork material', 'Formwork dimensions', 'Formwork design load', 'Formwork surface finish', 'Formwork release agent', 'Formwork tie system', 'Formwork insulation', 'Formwork stripping time', 'Formwork reuse', 'Formwork joints', 'Formwork bracing', 'Formwork accessories', 'Formwork stripping process', 'Formwork cleaning and maintenance', 'Formwork safety measures', 'Formwork compatibility', 'Formwork cost', 'Formwork sustainability'])

In [60]:
value_prompt = PromptTemplate.from_template(
    """I am an architect and want to describe building products in detail. I am looking to create a list of 20 accurate Values for an omniclass parameter. The omniclass we will be creating data for is {parameter} for the {product} omniclass.
    
    {format_instructions}""",
    partial_variables={'format_instructions': parameter_output_parser.get_format_instructions()}
)

In [61]:
value_chain = value_prompt | chat | parameter_output_parser

In [62]:
def get_values_for_parameter(parameter: str, product: str):
    values = value_chain.invoke({"parameter": parameter, "product": product})
    return values

In [63]:
from time import sleep

kv_columns = {}
for parameter in parameters.parameters:
    _values = get_values_for_parameter(parameter, product)
    kv_columns[parameter] = _values.parameters
    print(f"Finished {parameter}. Sleeping for 20 seconds.")
    sleep(20)

Finished Formwork type. Sleeping for 20 seconds.
Finished Formwork material. Sleeping for 20 seconds.
Finished Formwork dimensions. Sleeping for 20 seconds.
Finished Formwork reinforcement. Sleeping for 20 seconds.
Finished Formwork joints. Sleeping for 20 seconds.
Finished Formwork surface finish. Sleeping for 20 seconds.
Finished Formwork tie system. Sleeping for 20 seconds.
Finished Formwork release agent. Sleeping for 20 seconds.
Finished Formwork pressure. Sleeping for 20 seconds.
Finished Formwork stripping time. Sleeping for 20 seconds.
Finished Formwork curing method. Sleeping for 20 seconds.
Finished Formwork curing duration. Sleeping for 20 seconds.
Finished Formwork temperature control. Sleeping for 20 seconds.
Finished Formwork vibrations. Sleeping for 20 seconds.
Finished Formwork concrete mix design. Sleeping for 20 seconds.
Finished Formwork concrete pouring method. Sleeping for 20 seconds.
Finished Formwork concrete pouring sequence. Sleeping for 20 seconds.
Finished Fo

In [64]:
kv_columns

{'Formwork type': ['Adjustable Column Forms',
  'Aluminum Beam Gangs',
  'Aluminum Beam Joist',
  'Aluminum Beam Strongbacks',
  'Aluminum Beam Wall Forms',
  'Clamp Systems',
  'Column Clamps',
  'Column Forms',
  'Concrete Accessories',
  'Concrete Forms',
  'Concrete Formwork Accessories',
  'Form Braces',
  'Form Clips',
  'Form Hangers',
  'Form Liners',
  'Form Release Agents',
  'Form Ties',
  'Horizontal Panel Systems',
  'Panel Formwork',
  'Vertical Panel Systems',
  'Wall Forms'],
 'Formwork material': ['Aluminum Formwork',
  'Steel Formwork',
  'Plywood Formwork',
  'Timber Formwork',
  'Plastic Formwork',
  'Fabric Formwork',
  'Stay-In-Place Formwork',
  'Flexible Formwork',
  'Engineered Formwork',
  'Reusable Formwork',
  'Adjustable Formwork',
  'Modular Formwork',
  'Climbing Formwork',
  'Slipformwork',
  'Jump Formwork',
  'Gang Formwork',
  'Table Formwork',
  'Tunnel Formwork',
  'Flying Formwork',
  'Suspended Formwork',
  'Hydraulic Formwork'],
 'Formwork dimens

# Save as CSV File

In [65]:
fn = f'{product}.csv'

In [66]:
# get the `kv_columns` from above and save as a CSV file
with open(fn, 'w') as f:
    f.write(','.join(kv_columns.keys()) + '\n')
    for i in range(20):
        f.write(','.join([kv_columns[k][i] for k in kv_columns.keys()]) + '\n')